# Improvements to models

We attempt to modify the model in the following ways:  

- Adding restrictions to vision field of individuals (sensory regions, the regions where movement of other individuals (can be) are percepted)

- Generalizing to a second order model  (first order for leaders/predators and second order for followers/prey)

- Change the area of effect of other individuals: the interaction force is no longer averaged over the entire population but over $k$ nearest neighbours

<font size="1"> (Perhaps the first one is not much realistic as the herbivores generally have a large vision field to avoid predators and both of them can turn their heads to get a 360 view angle) </font>

(Draft) Now we can write the model as the following  

$$
\begin{cases}  
    \dot{x_i} = v_i \\  
    \dot{v_i} = a \ {\rm Repulsion \ average \ over \ KNN} + b \ {\rm Attraction \ average \ over \ KNN} + c \ {\rm Interaction \ with \ predator} \\  
    \dot{p} = d \ {\rm Interaction \ average \ over \ prey}
\end{cases}
$$

A useful thing to have:  

$$
R(x,x') = 
    \begin{cases} 
        \exp(-|x'-x|^{\gamma}) \frac{x'-x}{|x'-x|} & x' \in B_r(x) \setminus \{x\} \\
        0 & {\rm otherwise}    
    \end{cases}
$$  
which can be used to model the repulsion in acceleration term.